<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/Diplom4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install pynput

     |████████████████████████████████| 102kB 3.0MB/s 
     |████████████████████████████████| 184kB 16.8MB/s 
  Created wheel for evdev: filename=evdev-1.4.0-cp37-cp37m-linux_x86_64.whl size=97430 sha256=9341003ffc66bdf98bc999245398bff52b9a7301dfb5cfa69e04909d253e54b6
  Stored in directory: /root/.cache/pip/wheels/66/35/1d/38b2de1421ede48ebf0107faea56163d07059110b116a794a3
Successfully built evdev


In [6]:
!pip install patool

     |████████████████████████████████| 81kB 2.7MB/s 


In [7]:
import patoolib

In [8]:
archive = '/content/drive/My Drive/lenta-ru-news.rar'

In [ ]:
patoolib.extract_archive(archive)

patool: Extracting /content/drive/My Drive/lenta-ru-news.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/lenta-ru-news.rar"
patool:     with cwd='./Unpack_ohyzoe_z'


In [ ]:
corpora = pd.read_csv('lenta-ru-news.csv',engine='python', error_bad_lines=False)

In [ ]:
corpora.head()

In [ ]:
corpora['text'].head()

In [ ]:
import re

In [ ]:
new_corpora = []
for text in corpora['text']:
  split_regex = re.compile(r'[.|!|?|…]')
  sentences = filter(lambda t: t, [t.strip() for t in split_regex.split(str(text))])
  for sentence in sentences:
    new_corpora.append(sentence)
print(len(new_corpora))

In [ ]:
new_corpora[0:10]

In [ ]:
!pip install ufal.udpipe

In [ ]:
!pip install wget

In [ ]:
import wget
import sys
udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
modelfile = wget.download(udpipe_url)

In [ ]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
   entities = {'PROPN'}
   named = False  # переменная для запоминания того, что нам встретилось имя собственное
   memory = []
   mem_case = None
   mem_number = None
   tagged_propn = []
# обрабатываем текст, получаем результат в формате conllu:
   processed = pipeline.process(text)
# пропускаем строки со служебной информацией:
   content = [l for l in processed.split('\n') if not l.startswith('#')]
# извлекаем из обработанного текста леммы, тэги и морфологические характеристики
   tagged = [w.split('\t') for w in content if w]
   for t in tagged:
       if len(t) != 10: # если список короткий — строчка не содержит разбора, пропускаем
           continue
       (word_id,token,lemma,pos,xpos,feats,head,deprel,deps,misc) = t 
       if not lemma or not token: # если слово пустое — пропускаем
           continue
       if pos in entities: # здесь отдельно обрабатываем имена собственные — они требуют особого обращения
           if '|' not in feats:
               tagged_propn.append('%s_%s_%s_%s_%s_%s' % (word_id, token, lemma, pos, head, deprel))
               continue
           morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
           if 'Case' not in morph or 'Number' not in morph:
               tagged_propn.append('%s_%s_%s_%s_%s_%s' % (word_id, token, lemma, pos, head, deprel))
               continue
           if not named:
               named = True
               mem_case = morph['Case']
               mem_number = morph['Number']
           if morph['Case'] == mem_case and morph['Number'] == mem_number:
               memory.append(lemma)
               if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                   named = False
                   past_lemma = '::'.join(memory)
                   memory = []
                   tagged_propn.append(past_lemma + '_PROPN ')
           else:
               named = False
               past_lemma = '::'.join(memory)
               memory = []
               tagged_propn.append(past_lemma + '_PROPN ')
               tagged_propn.append('%s_%s_%s_%s_%s_%s' % (word_id, token, lemma, pos, head, deprel))
       else:
           if not named:
               tagged_propn.append('%s_%s_%s_%s_%s_%s' % (word_id, token, lemma, pos, head, deprel))
           else:
               named = False
               past_lemma = '::'.join(memory)
               memory = []
               tagged_propn.append(past_lemma + '_PROPN ')
               tagged_propn.append('%s_%s_%s_%s_%s_%s' % (word_id, token, lemma, pos, head, deprel))
   if not keep_punct: # обрабатываем случай, когда пользователь попросил не сохранять пунктуацию (по умолчанию она сохраняется)
       tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
   if not keep_pos:
       tagged_propn = [word.split('_')[0] for word in tagged_propn]
   return tagged_propn

In [ ]:
from ufal.udpipe import Model, Pipeline
import os
import re
def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
   udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
   udpipe_filename = udpipe_model_url.split('/')[-1]
   if not os.path.isfile(modelfile):
       print('UDPipe model not found. Downloading...', file=sys.stderr)
       wget.download(udpipe_model_url)
   #print('\nLoading the model...', file=sys.stderr)
   model = Model.load(modelfile)
   process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
   #print('Processing input...', file=sys.stderr)
   lines = text.split('\n')
   tagged = []
   for line in lines:
# line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
       output = process(process_pipeline, text=line)
       tagged_line = ' '.join(output)
       tagged.append(tagged_line)
   return '\n'.join(tagged)
text = new_corpora[100000:120000]
for sentence in text:
   processed_text = tag_ud(text=sentence, modelfile=modelfile)
   #print(processed_text[:350])
   with open('my_text.txt', 'a', encoding='utf-8') as out:
       out.write(processed_text + '\n')

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
new_corpora = open("my_text.txt", encoding="UTF-8", errors='ignore')

In [ ]:
tokens=[]
for sentence in new_corpora:
  tokens.append(nltk.word_tokenize(sentence))

In [ ]:
tokens[0:10]

In [ ]:
law_corpora=[]
for sentence in tokens:
  for token in sentence:
    if "_суд_" in token:
      if sentence not in law_corpora:
        law_corpora.append(sentence)
    elif "_судья_" in token:
      if sentence not in law_corpora:
        law_corpora.append(sentence)
    elif "_адвокат_" in token:
      if sentence not in law_corpora:
        law_corpora.append(sentence)
    elif "_прокурор_" in token:
      if sentence not in law_corpora:
        law_corpora.append(sentence)
    else:
      pass
print(len(law_corpora))

In [ ]:
law_corpora[0:10]

In [ ]:
with open("law_corpora.txt", "w") as file:
  print(law_corpora, file=file, sep="\n")
file.close()

In [ ]:
count=0
for sentence in tokens:
  count += len(sentence)
print(count)

In [ ]:
count=0
for sentence in law_corpora:
  count += len(sentence)
print(count)

In [ ]:
len(tokens)